In [ ]:
!pip install transformers==4.27
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.8 MB/s eta 0:00:

In [12]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
import warnings
warnings.filterwarnings(action='ignore')

In [14]:
from datasets import load_dataset
dataset = load_dataset('nsmc')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [27]:
train_data= dataset['train'].shuffle(seed=42).select(range(2000))
test_data = dataset['test'].shuffle(seed=42).select(range(2000))

In [28]:
print(train_data)

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 2000
})


In [26]:
print(test_data)

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 2000
})


In [29]:
model_name = 'bert-base-multilingual-cased'

In [32]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [33]:
tokenizer.tokenize(train_data['document'][0])

['For',
 'Carl',
 '.',
 '칼',
 '세',
 '##이',
 '##건',
 '##으로',
 '시',
 '##작',
 '##해서',
 '칼',
 '세',
 '##이',
 '##건',
 '##으로',
 '끝',
 '##난',
 '##다',
 '.']

In [38]:
train_encoding= tokenizer(train_data['document'],return_tensors='pt',padding=True, truncation=True)
test_encoding=  tokenizer(test_data['document'],return_tensors='pt',padding=True, truncation=True)

In [65]:
from torch.utils.data import Dataset

class EmotionDataset(Dataset):
  def __init__(self,encodings,labels):
    self.encoding=encodings
    self.labels=labels
  def __getitem__(self,idx):
    data={key:val[idx] for key, val in self.encoding.items()}
    data['labels']=torch.tensor(self.labels[idx]).long()
    return data
  def __len__(self):
    return len(self.labels)

In [66]:
train_set = EmotionDataset(train_encoding,train_data['label'])
test_set = EmotionDataset(test_encoding,test_data['label'])

In [67]:
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [68]:
from datasets import load_metric
def compute_metrics(pred):
  labels=pred.label_ids
  preds=pred.predictions.argmax(-1)

  m1=load_metric('accuracy')
  m2=load_metric('f1')

  acc=m1.compute(predictions=preds, references=labels)['accuracy']
  f1=m2.compute(predictions=preds, references=labels)['f1']

  return {'accuracy':acc, 'f1':f1}

In [69]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir='./outputs',
    logging_dir = './logs',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=50,
    save_steps=50,
    save_total_limit=2
)

In [70]:
model.to(device)
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics
)

In [71]:
trainer.train()

Step,Training Loss
50,0.706600
100,0.672400
150,0.543200
200,0.425700
250,0.317900
300,0.211200
350,0.194300


Step,Training Loss
50,0.706600
100,0.672400
150,0.543200
200,0.425700
250,0.317900
300,0.211200
350,0.194300
400,0.120100
450,0.108900
500,0.060400


TrainOutput(global_step=630, training_loss=0.2757026105646103, metrics={'train_runtime': 562.8593, 'train_samples_per_second': 35.533, 'train_steps_per_second': 1.119, 'total_flos': 1223055296400000.0, 'train_loss': 0.2757026105646103, 'epoch': 10.0})

In [72]:
trainer.evaluate()

{'eval_loss': 1.19325590133667,
 'eval_accuracy': 0.769,
 'eval_f1': 0.7657200811359026,
 'eval_runtime': 17.8774,
 'eval_samples_per_second': 111.873,
 'eval_steps_per_second': 3.524,
 'epoch': 10.0}